Thanks to abhishek
Please upvote this

https://www.kaggle.com/abhishek/beating-the-benchmark-neural-network

In [ ]:
import os
import gc
import joblib
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics, preprocessing
import xgboost as xgb
import tensorflow as tf
from keras.layers import Dense, Input
from collections import Counter
from keras.layers import *
from keras.optimizers import Adam
from keras.models import Model, load_model
from keras import callbacks
from keras import backend as K
from keras.layers import Dropout
from sklearn.preprocessing import LabelEncoder
import warnings
from keras.layers import ReLU
from keras.layers import Input, Reshape, concatenate, Dense, GlobalAveragePooling1D, BatchNormalization, Flatten, \
    Dropout, PReLU, LeakyReLU, Embedding,SpatialDropout1D
from keras.layers import ReLU

warnings.filterwarnings("ignore")

In [ ]:
def fallback_auc(y_true, y_pred):
    try:
        return metrics.roc_auc_score(y_true, y_pred)
    except:
        return 0.5

In [ ]:
def auc(y_true, y_pred):
    return tf.py_function(fallback_auc, (y_true, y_pred), tf.double)

In [ ]:
NFOLDS = 5
RANDOM_STATE = 42

In [ ]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [ ]:
y = train.target
ids = train.id.values
#train = train.drop(['id', 'target'], axis=1)
train_cat = train[['wheezy-copper-turtle-magic']]
test_ids = test.id.values
#test = test[train.columns]
test_cat = test[['wheezy-copper-turtle-magic']]

folds = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=RANDOM_STATE)
oof_preds = np.zeros((len(train)))
test_preds = np.zeros((len(test)))

In [ ]:
#转化率 target_encoding and count
def ratio(df):
    r = df.groupby(['wheezy-copper-turtle-magic'])['target'].mean().reset_index()
    r.columns = ['wheezy-copper-turtle-magic','ratio']
    return r

In [ ]:
for fold_, (trn_, val_) in enumerate(folds.split(y, y)):
    print("Current Fold: {}".format(fold_))
    r = ratio( train.iloc[val_])
    train_add_f = train.merge(r, on = ['wheezy-copper-turtle-magic'], how='left')
    test_add_f = test.merge(r, on = ['wheezy-copper-turtle-magic'], how='left')
    
    feature = [i for i in train.columns if i not in ['id', 'target','wheezy-copper-turtle-magic']]
    raw_feature = ['ratio']
    
    trn_x, trn_y = train_add_f[feature].iloc[trn_], y.iloc[trn_]
    val_x, val_y = train_add_f[feature].iloc[val_], y.iloc[val_]
    
    raw_train = train_add_f[raw_feature].iloc[trn_]
    raw_valid = train_add_f[raw_feature].iloc[val_]
    raw_test = test_add_f[raw_feature]
    
    scl = preprocessing.StandardScaler()
    scl.fit(pd.concat([trn_x,val_x, test[feature]]))
    trn_x = scl.transform(trn_x)
    val_x = scl.transform(val_x)
    tst_x = scl.transform(test[feature])
    
    lb = LabelEncoder().fit(pd.concat([train_cat,test_cat]))
    
    train_c = lb.transform(train_cat.iloc[trn_])
    val_c = lb.transform(train_cat.iloc[val_])
    test_c = lb.transform(test_cat)
    
    inp = Input(shape=(trn_x.shape[1],))
    inp_cat = Input(shape=(1,))
    inp_f = Input(shape=(len(raw_feature),))
    emb = Embedding(1024, 256, trainable= True)(inp_cat)
    emb = SpatialDropout1D(0.1)(emb)
    
    x = Dense(1024, activation="linear")(inp)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Dropout(0.3)(x)
    x = Dense(1024, activation="linear")(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Dropout(0.3)(x)
    c = concatenate([ Flatten()(emb),  Flatten()(emb),  Flatten()(emb),  Flatten()(emb)])
    s = Subtract()([x,c])
    m = Multiply()([x,c])
    x = concatenate([x, Flatten()(emb), inp_f, s, m])
    x = Dense(1024, activation="linear")(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Dropout(0.2)(x)
    out = Dense(1, activation="sigmoid")(x)
    clf = Model(inputs=[inp,inp_cat, inp_f],outputs=out)
    clf.compile(loss='binary_crossentropy', optimizer='adam', metrics=[auc])

    es = callbacks.EarlyStopping(monitor='val_auc', min_delta=0.001, patience=10,
                                 verbose=1, mode='max', baseline=None, restore_best_weights=False)

    rlr = callbacks.ReduceLROnPlateau(monitor='val_auc', factor=0.5,
                                      patience=3, min_lr=1e-6, mode='max', verbose=1)

    chk = callbacks.ModelCheckpoint('model.h5', monitor='val_auc', verbose=1, save_best_only=True,
                                    save_weights_only=False, mode='max', period=1)

    clf.fit([trn_x,train_c, raw_train], trn_y, validation_data=([val_x,val_c, raw_valid], val_y), callbacks=[es, rlr, chk], epochs=100, batch_size=1024)
    
    clf = load_model('model.h5', custom_objects={"auc": auc})
    val_preds = clf.predict([val_x,val_c, raw_valid])
    test_fold_preds = clf.predict([tst_x, test_c, raw_test])
    
    print("AUC = {}".format(metrics.roc_auc_score(val_y, val_preds)))
    oof_preds[val_] = val_preds.ravel()
    test_preds += test_fold_preds.ravel()
    
    K.clear_session()
    gc.collect()

In [ ]:
print("Saving OOF predictions")
oof_preds = pd.DataFrame(np.column_stack((ids, oof_preds)), columns=['id', 'target'])
oof_preds.to_csv('oof_preds.csv', index=False)
print("Overall AUC={}".format(metrics.roc_auc_score(y.values, oof_preds.target.values)))

In [ ]:
test_preds /= NFOLDS
print("Saving submission file")
submission = pd.DataFrame.from_dict({
    'id': test_ids,
    'target': test_preds
})

In [ ]:
sample = pd.read_csv("../input/sample_submission.csv", usecols=["id"])
sample = sample.merge(submission, on="id", how="left")
sample.to_csv("submission.csv", index=False)

In [ ]:
sample.head()